**TODO:**

- include_top, false
- finetune with trainable layers, true
- keep images large

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import keras
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Conv2D, Dropout
from keras.layers import GlobalAveragePooling2D, BatchNormalization

from squeezenet import SqueezeNet, preprocess_input

Using TensorFlow backend.


In [3]:
from matplotlib import pyplot as plt
import time
import helper_keras
import numpy as np
import pandas as pd
import os

/home/jason/anaconda3/envs/goodscifi/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [4]:
# Make sure tensorflow is using GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14682023978636540792
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 7862963405
locality {
  bus_id: 1
}
incarnation: 1467946150659410101
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0"
]


**Data Paths**

In [ ]:
notbook_path = '/home/jason/DeepLearning/github/goodscifi/development/notebooks/'
home_path = '/home/jason/DeepLearning/github/goodscifi/development/'
data_path = '/home/jason/DeepLearning/github/goodscifi/development/data/'
model_path = '/home/jason/DeepLearning/github/goodscifi/development/models/'

In [ ]:
# Data is imbalance try using unsamples found in train_undersample/
training_data = data_path+'dataset/books/train/'
validation_data = data_path+'dataset/books/valid/'

**Input Data**

In [ ]:
# Image Preprocessing and data augmentation
image_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
#                                horizontal_flip=True
                              )

In [ ]:
train_batches_aug = helper_keras.get_batches(training_data,
                                            gen=image_gen, 
                                            batch_size=128,
                                            target_size=(227,227)) 
val_batches_aug = helper_keras.get_batches(validation_data,
                                            gen=image_gen, 
                                            batch_size=128,
                                            target_size=(227,227))

In [ ]:
# Set steps per epoch for train & validation data
steps_per_epoch = train_batches_aug.samples/train_batches_aug.batch_size
val_steps = val_batches_aug.samples/val_batches_aug.batch_size

print('steps per epoch:', steps_per_epoch)
print('validation steps:', val_steps)

In [ ]:
cls_weight = helper_keras.get_class_ratio(training_data); cls_weight

**Func**

In [ ]:
# TODO: Flatten vs GlobalAveragePooling2D() in this context
def add_new_last_layer(base_model,classes=2):
#     x = base_model.output
#     x = GlobalAveragePooling2D()(x)
#     x = Dense(512)(x)
#     x = Activation('relu')(x)
#     x = Dense(classes)(x)
    
    x = base_model.output
    x = Conv2D(classes, (1, 1))(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D()(x)

    predictions = Activation('softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [ ]:
def setup_finetuning(model, num_layers_to_freeze):
    for layer in model.layers[:num_layers_to_freeze]:
        layer.trainable = False
    for layer in model.layers[num_layers_to_freeze:]:
        layer.trainable = True

In [ ]:
# Add to helper
def setup_transfer_learning(base_model, model):
    for layer in base_model.layers:
        layer.trainable = False

In [ ]:
# Add to helper
def create_callbacks(run_label):
    outputFolder = model_path+'squeezenet/'
    if not os.path.exists(outputFolder):
        os.makedirs(outputFolder)
    filepath=outputFolder+"weights.best.hdf5"
    
    # Checkpoint callback
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0,
                                 save_best_only=True, save_weights_only=True)

    # Early stopping callback
    earlystop = EarlyStopping(monitor='val_loss', patience=40,verbose=1, mode='auto')
    
    # Tensorboard callback
    tensorboard = TensorBoard(log_dir='./logs/squeeze_'+str(run_label))
    
    return [checkpoint, tensorboard]

**Training**

In [ ]:
base_model = SqueezeNet(weights='imagenet', include_top=False, pooling='avg')
base_model.summary()

In [ ]:
setup_finetuning(base_model, 47) # 54, 47, 40
for layer in base_model.layers:
    if layer.trainable == True:
        print(layer.name)

In [ ]:
lr_to_test = [1e-06]
cls_weight = {0:8, 1:1}
transfer_learn = False
r_label = 'run1_1e-06'

# trainable layers [9, 8, 7]
layers_to_freeze = [47]

# Loop over learning rates
for ltf in layers_to_freeze:
    print('\n\nTesting model with ltf parameter: %f\n'%ltf )
    
    # Set run label
    run_label = r_label+'_ltf-'+str(ltf)
    
    # Create new model
    base_model = SqueezeNet(weights='imagenet', include_top=False, pooling=None)
    base_model = Model(inputs=base_model.input, outputs=base_model.output)
    model = add_new_last_layer(base_model)
    
    if transfer_learn:
        setup_transfer_learning(base_model, model)
    else:
        setup_finetuning(model, ltf)
    
    # Optimize
    # Use SGD to start - find the best appropriate lr for Adam
    optimizer = Adam(lr=lr_to_test)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Callbacks
    callbacks_list = create_callbacks(run_label)

    # Fit the model
    start = time.time()
    print("Model started training at",time.ctime())
    history =  model.fit_generator(train_batches_aug,
                                    steps_per_epoch=steps_per_epoch,
                                    validation_data=val_batches_aug,
                                    validation_steps=val_steps,
                                    epochs=500,
                                    verbose=0,
                                    callbacks= callbacks_list,
                                    class_weight=cls_weight)
    end = time.time()
    print("Model took %0.2f seconds to train"%(end - start))
    # plot model history
    helper_keras.plot_model_history(history)
    # Save model+weights(last known configs)
    model.save(model_path+'squeeze_'+str(run_label)+'.h5')

**Save Model**

In [ ]:
# optimizer = Adam(lr=0.000000001)
# model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_path = '/home/jason/DeepLearning/github/goodscifi/development/models/squeeze_run6_1e-06_ltf-47.h5'
model = keras.models.load_model(model_path)

In [ ]:
# model.summary()

In [ ]:
# run 7 at batch_size 128, no data augmentation
run_label = 'run7_1e-06_ltf-47'
callbacks_list = create_callbacks(run_label)

history =  model.fit_generator(train_batches_aug,
                                steps_per_epoch=steps_per_epoch,
                                validation_data=val_batches_aug,
                                validation_steps=val_steps,
                                epochs=500,
                                verbose=0,
                                callbacks=callbacks_list,
                                class_weight=cls_weight)

model.save(model_path+'squeeze_'+str(run_label)+'.h5')